# PREDICTION OF AUTOMOTIVE ACCIDENT SEVERITY

## TABLE OF CONTENT
* [Uploading libraries and packages](#UPLOADING-LIBRARIES-AND-PACKAGES)
* [Data preprocessing](#DATA-PREPROCESSING)
* [Re-sampling](#RE-SAMPLING)
* [Standardization and PCA analysis](#STANDARDIZATION-AND-PCA-ANALYSIS)
* [Models](#DECISION-TREE)
    * [Deciosion Tree](#DECISION-TREE)
    * [Neural Network](#NEURAL-NETWORK)
    * [KNN](#KNN)
    * [Logistic Regression](#LOGISTIC-REGRESSION)
* [Summary](#SUMMARY)

## UPLOADING LIBRARIES AND PACKAGES

* GENERAL

In [4]:
import numpy as np
import pandas as pd

* RESAMPLING

In [6]:
import matplotlib.pyplot as plt   
from pydotplus import graph_from_dot_data
from IPython.display import Image  
import seaborn as sns
from IPython.display import HTML, display
import tabulate

ModuleNotFoundError: No module named 'pydotplus'

* PLOTS, TABS

In [7]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

ModuleNotFoundError: No module named 'imblearn'

* CROSS- VALIDATION

In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

* DECISION TREE

In [9]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz

* STANDARDIZATION, PCA ANALYSIS

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

* NEURAL NETWORK

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.python.keras import utils
from sklearn.preprocessing import LabelEncoder
#from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'tensorflow'

* KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier

* LOGISTIC REGRESSION

In [14]:
from sklearn.linear_model import LogisticRegression

## DATA PREPROCESSING 

In [15]:
#reading datafiles
df1 = pd.read_csv('data/2014 excel.csv')
df2 = pd.read_csv('data/2015 excel.csv')
df3 = pd.read_csv('data/2016 excel.csv')

#merging datafiles
frames = [df1, df2, df3]
result = pd.concat(frames, axis=0)

FileNotFoundError: File b'data/2014 excel.csv' does not exist

In [16]:
#dropping unnecessary columns
result= result.drop(['Grid Ref: Northing','Grid Ref: Easting','Reference Number','Expr1','Accident Date','Lighting Conditions','1st Road Class'],axis=1)

#dropping observations with missing values
result= result.dropna(axis=0)

NameError: name 'result' is not defined

In [17]:
#listing objects for the further processing 
list1=['Monday','Tuesday','Wednesday','Thursday']
list2=['Friday','Saturday','Sunday']
list3=['Fine without high winds','Fine with high winds']
list4=['Snowing without high winds','Snowing with high winds']
list5=['Raining without high winds','Raining with high winds']
list6=['Fog or mist']
list7=['Minibus (8 to 16 passenger seats)','Bus or coach (17 or more passenger seats)']
list8=['Goods vehicle 3.5 tonnes mgw and under','Goods vehicle over 3.5 tonnes and under 7.5 tonnes mgw','goods','Goods vehicle over 3.5 tonnes and under 7.5 tonnes mgw']
list9=['Motorcycle over 50cc and up to 125cc','Motorcycle over 125cc and up to 500cc','M cycle 50cc and under','Motorcycle over 500cc','Motorcycle Unknown CC']
list10=['Car','Taxi or Private hire car']
list11=['Driver', 'Driver ']
list12=['Pedestrian']

In [18]:
#converting 'Day' feature
def b(row):
    if row['Day']in list1:
        val='weekday'
    elif row['Day']in list2:
        val='weekend'
    return val
result['Day'] = result.apply(b, axis=1)

#converting 'Weather Conditions' feature
def e(row):
    if row['Weather Conditions']in list3:
        val='fine'
    elif row['Weather Conditions']in list4:
        val='snowing'
    elif row['Weather Conditions']in list5:
        val='raining'
    elif row['Weather Conditions']in list6:
        val='fog'
    else: 
        val='other'   
    return val
result['Weather Conditions'] = result.apply(e, axis=1)

#converting 'Type of Vehicle' feature
def j(row):
    if row['Type of Vehicle']in list7:
        val='bus'
    elif row['Type of Vehicle']in list8:
        val='goods vehicle'
    elif row['Type of Vehicle']in list9:
        val='motorcycle'
    elif row['Type of Vehicle']in list10:
        val='car'
    else: 
        val='other'
    return val
result['Type of Vehicle'] = result.apply(j, axis=1)

#converting 'Time (24hr) feature
def c(row):
    if 600< row['Time (24hr)'] <= 1801:
        val='Day-time'
    else:
        val='Night-time'
    return val
result['Time (24hr)'] = result.apply(c, axis=1)

#converting 'Casualty Class' feature
def a(row):
    if row['Casualty Class']in list11:
        val='Driver'
    elif row['Casualty Class']in list12:
        val='Pedestrian'
    else: 
        val='Passenger'
    return val
result['Casualty Class'] = result.apply(a, axis=1)

#creating dummy variables for categegorical variales 
result=pd.get_dummies(result,columns=['Day','Time (24hr)','Road Surface','Weather Conditions','Casualty Class','Sex of Casualty','Type of Vehicle'])

#dealing with columns containing the same information
result = result.drop(['Day_weekday', 'Time (24hr)_Day-time', 'Sex of Casualty_Female'] , axis=1)

NameError: name 'result' is not defined